In [13]:
import numpy as np
import xarray as xr
from netCDF4 import Dataset
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import os
import glob
import re

In [ ]:
folder_paths = ['/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_094824', '/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_104942', '/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_102335']

chlor_a_data_list = []

for folder_path in folder_paths:
    print("new folder")
    nc_files = [f for f in os.listdir(folder_path) if f.endswith('.nc')]
    for file_name in nc_files:
        file_path = os.path.join(folder_path, file_name)
        
        dataset = xr.open_dataset(file_path)

        chlor_a_data_list.append(dataset['chlor_a'])

        dataset.close()

print("went through all files")

new folder


In [5]:
chunk_size = 10
n_chunks = int(np.ceil(len(chlor_a_data_list) / chunk_size))

print(n_chunks)


109


In [6]:
concatenated_result1 = None
for i in range(0,36):
    chunk = chlor_a_data_list[i * chunk_size: (i + 1) * chunk_size]

    chunk_concat = xr.concat(chunk, dim='time')

    if concatenated_result1 is None:
        concatenated_result1 = chunk_concat
    else:
        concatenated_result1 = xr.concat([concatenated_result1, chunk_concat], dim='time')

    print(f"Chunk {i + 1} concatenation completed.")

Chunk 1 concatenation completed.
Chunk 2 concatenation completed.
Chunk 3 concatenation completed.
Chunk 4 concatenation completed.
Chunk 5 concatenation completed.
Chunk 6 concatenation completed.
Chunk 7 concatenation completed.
Chunk 8 concatenation completed.
Chunk 9 concatenation completed.
Chunk 10 concatenation completed.
Chunk 11 concatenation completed.
Chunk 12 concatenation completed.
Chunk 13 concatenation completed.
Chunk 14 concatenation completed.
Chunk 15 concatenation completed.
Chunk 16 concatenation completed.
Chunk 17 concatenation completed.
Chunk 18 concatenation completed.
Chunk 19 concatenation completed.
Chunk 20 concatenation completed.
Chunk 21 concatenation completed.
Chunk 22 concatenation completed.
Chunk 23 concatenation completed.
Chunk 24 concatenation completed.
Chunk 25 concatenation completed.
Chunk 26 concatenation completed.
Chunk 27 concatenation completed.
Chunk 28 concatenation completed.
Chunk 29 concatenation completed.
Chunk 30 concatenation 

In [ ]:
output_file1 = '/cluster/home/maikents/satellite_data/chunks/chunk1.nc'
concatenated_result1.to_netcdf(output_file1)

In [ ]:
concatenated_result2 = None
for i in range(37,72):
    chunk = chlor_a_data_list[i * chunk_size: (i + 1) * chunk_size]

    chunk_concat = xr.concat(chunk, dim='time')

    if concatenated_result2 is None:
        concatenated_result2 = chunk_concat
    else:
        concatenated_result2 = xr.concat([concatenated_result2, chunk_concat], dim='time')

    print(f"Chunk {i + 1} concatenation completed.")

In [ ]:
output_file2 = '/cluster/home/maikents/satellite_data/chunks/chunk2.nc'
concatenated_result2.to_netcdf(output_file2)

In [ ]:
concatenated_result3 = None
for i in range(73,109):
    chunk = chlor_a_data_list[i * chunk_size: (i + 1) * chunk_size]

    chunk_concat = xr.concat(chunk, dim='time')

    if concatenated_result3 is None:
        concatenated_result3 = chunk_concat
    else:
        concatenated_result3 = xr.concat([concatenated_result3, chunk_concat], dim='time')

    print(f"Chunk {i + 1} concatenation completed.")

In [ ]:
concatenated_result = xr.concat([concatenated_result1, concatenated_result2, concatenated_result3], dim='time')

In [ ]:
chl_time_avg = concatenated_result.mean(dim='time')
print("averaged data")


In [ ]:
output_file = '/cluster/home/maikents/satellite_data/chlorophyll_time_avg.nc'
chl_time_avg.to_netcdf(output_file)


print(f"Time-averaged chlorophyll data saved to {output_file}")

"""
The output of the time-averaged chlorophyll data will look like this:
Dimensions: (lat, lon)
Data: Each point on the grid will have the average chlorophyll concentration over the period you’re averaging (in this case, across three years).
"""